In [1]:
import pandas as pd
import numpy as np
import ast

In [6]:
movies = pd.read_csv("Dataset/movies_dataset.csv")

C:\Users\Mauricio Figueroa\AppData\Local\Temp\ipykernel_11020\2117833795.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("Dataset/movies_dataset.csv")


In [116]:
# Queremos ver el formato de los campos (son 24)
movies.iloc[:, :10].head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ..."


In [117]:
movies.iloc[:, 10:20].head(1)

,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline
0,21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN


In [118]:
movies.iloc[:, 20:].head(1)

,title,video,vote_average,vote_count
0,Toy Story,False,7.7,5415.0


In [119]:
movies.shape

(45466, 24)

Datos anidados: "belongs_to_collection", "genres", "production_companies", "production_countries", "spoken_languages"

In [120]:
# Definir una función que recorra una lista de diccionarios de acuerdo a una etiqueta particular y cree una lista de las categorias de cada fila
# (válido para multilabel)

def dict_to_lista(columna, tag):
    lista = pd.Series(data = [])
    lista_unique = []
    for index, fila in movies[columna].items():
        if isinstance(fila, str):
            try:
                fila = ast.literal_eval(fila)
                lista_aux = []
                if isinstance(fila, list):
                    if len(fila) > 0:
                        for item in fila:
                            if isinstance(item, dict):
                                if tag in item:
                                    lista_aux.append(item[tag])
                                    lista.loc[index] = lista_aux
                                    if item[tag] not in lista_unique:
                                        lista_unique.append(item[tag])
                    else:
                        lista.loc[index] = ["Sin_Categoria"]
                elif isinstance(fila, dict):
                    lista_aux.append(fila[tag])
                    lista.loc[index] = lista_aux
                    if fila[tag] not in lista_unique:
                        lista_unique.append(fila[tag])
                if isinstance(fila, float):
                    lista.loc[index] = ["Sin_Categoria"]
            except ValueError:
                print(f"Error al procesar la fila: {fila}")
        else:
            lista.loc[index] = ["Sin_Categoria"]
    return lista, lista_unique

In [121]:
# Creamos un dataframe movies_aux replica del original y 
# corregimos tres filas que encontramos estan corridas 9 columnas (hacer un shift)
movies_aux = movies
indices_conflictivos = [19730, 29503, 35587]
movies_aux.loc[indices_conflictivos] = movies_aux.loc[indices_conflictivos].shift(periods = 9, axis = 1)

In [122]:
# Borramos las columnas anidadas de movies_aux porque van a ser reemplazadas por las transformadas
movies_aux = movies_aux.drop(columns = ["belongs_to_collection", "genres", "production_companies", "production_countries", "spoken_languages"])

In [123]:
# Con nuestra función creada, hacemos la transformación de dichas columnas y las cargamos en un df auxiliar. También guardamos los unique para analizarlos
movies_aux["belongs_to_collection"], belongs_to_collection_unique = dict_to_lista("belongs_to_collection", "name")
movies_aux["genres"], genres_unique = dict_to_lista("genres", "name")
movies_aux["production_companies"], production_companies_unique = dict_to_lista("production_companies", "name")
movies_aux["production_countries"], production_countries_unique = dict_to_lista("production_countries", "name")
movies_aux["spoken_languages"], spoken_languages_unique = dict_to_lista("spoken_languages", "name")

C:\Users\Mauricio Figueroa\AppData\Local\Temp\ipykernel_15312\4112953925.py:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lista = pd.Series(data = [])
C:\Users\Mauricio Figueroa\AppData\Local\Temp\ipykernel_15312\4112953925.py:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lista = pd.Series(data = [])
C:\Users\Mauricio Figueroa\AppData\Local\Temp\ipykernel_15312\4112953925.py:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lista = pd.Series(data = [])
C:\Users\Mauricio Figueroa\AppData\Local\Temp\ipykernel_15312\4112953925.py:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version.

In [124]:
# Vemos si hay coherencia en las categorías de cada columna
print("Número de colecciones distintas:", len(belongs_to_collection_unique))
print(belongs_to_collection_unique)

Número de colecciones distintas: 1695
['Toy Story Collection', 'Grumpy Old Men Collection', 'Father of the Bride Collection', 'James Bond Collection', 'Balto Collection', 'Ace Ventura Collection', 'Chili Palmer Collection', 'Babe Collection', 'Mortal Kombat Collection', 'Pocahontas Collection', 'The Lawnmower Man Collection', 'Friday Collection', 'From Dusk Till Dawn Collection', 'Screamers Collection', 'The Muppet Collection', 'The Neverending Story Collection', 'Bad Boys Collection', 'Batman Collection', 'Brooklyn Cigar Store Collection', 'Casper Collection', 'Mexico Trilogy', 'Die Hard Collection', 'Teenage Apocalypse Trilogy', 'Free Willy Collection', 'Power Rangers Collection', 'Síť (kolekce)', 'The Prophecy Collection', 'Showgirls Collection', 'Species Collection', 'Under Siege Collection', 'Burnt by the Sun Trilogy', 'Before... Collection', 'Clerks Collection', 'Dumb and Dumber Collection', 'A Goofy Movie Collection', 'The Vampire Chronicles', 'Star Wars Collection', 'Nemesis Co

In [125]:
print("Número de géneros distintos:", len(genres_unique))
print(genres_unique)

Número de géneros distintos: 20
['Animation', 'Comedy', 'Family', 'Adventure', 'Fantasy', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'History', 'Science Fiction', 'Mystery', 'War', 'Foreign', 'Music', 'Documentary', 'Western', 'TV Movie']


In [126]:
# Había valores erróneos en la columna género (), que correspondian a productoras. Debemos ponerlos en la columna adecuada
# ['Carousel Productions', 'Vision View Entertainment', 'Telescene Film Group Productions', 'Aniplex', 'GoHands', 'BROSTA TV', 'Mardock Scramble Production Committee', 'Sentai Filmworks', 'Odyssey Media', 'Pulser Productions', 'Rogue State', 'The Cartel']
generos_erroneos = genres_unique[20:]
indices_conflictivos = []
for index, generos in movies_aux["genres"].items():
    for genero in generos:
        if genero in generos_erroneos:
            if index not in indices_conflictivos:
                indices_conflictivos.append(index)

In [127]:
movies_aux.loc[indices_conflictivos]        # Aqui veiamos que estaban corridas 9 columnas las filas [19730, 29503, 35587]
                                            # Para corregir este error debemos hacer un shift antes de generar las columnas

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,tagline,title,video,vote_average,vote_count,belongs_to_collection,genres,production_companies,production_countries,spoken_languages


In [128]:
print("Número de productoras distintas:", len(production_companies_unique))
print(production_companies_unique)

Número de productoras distintas: 23538
['Pixar Animation Studios', 'TriStar Pictures', 'Teitler Film', 'Interscope Communications', 'Warner Bros.', 'Lancaster Gate', 'Twentieth Century Fox Film Corporation', 'Sandollar Productions', 'Touchstone Pictures', 'Regency Enterprises', 'Forward Pass', 'Paramount Pictures', 'Scott Rudin Productions', 'Mirage Enterprises', 'Constellation Entertainment', 'Worldwide', 'Mont Blanc Entertainment GmbH', 'Walt Disney Pictures', 'Universal Pictures', 'Imperial Entertainment', 'Signature Entertainment', 'United Artists', 'Eon Productions', 'Columbia Pictures', 'Castle Rock Entertainment', 'Enigma Pictures', 'Amblin Entertainment', 'Amblimation', 'Hollywood Pictures', 'Cinergi Pictures Entertainment', 'Le Studio Canal+', 'Laurence Mark Productions', 'Metro-Goldwyn-Mayer (MGM)', 'Carolco Pictures', 'Légende Entreprises', 'Syalis DA', 'De Fina-Cappa', 'Columbia Pictures Corporation', 'Miramax Films', 'A Band Apart', 'O Entertainment', 'Morgan Creek Product

In [129]:
print("Número de países distintos:", len(production_countries_unique))
print(production_countries_unique)

Número de países distintos: 160
['United States of America', 'Germany', 'United Kingdom', 'France', 'Italy', 'Spain', 'China', 'Australia', 'South Africa', 'Canada', 'Switzerland', 'Belgium', 'Japan', 'Iran', 'Netherlands', 'Hong Kong', 'Tunisia', 'Ireland', 'Dominican Republic', 'Croatia', 'Russia', 'Macedonia', 'Austria', 'Taiwan', 'New Zealand', 'Mexico', 'Poland', 'Peru', 'Cuba', 'Liechtenstein', 'Denmark', 'Portugal', 'Finland', 'Sweden', 'Argentina', 'Iceland', 'South Korea', 'Serbia', 'Hungary', 'Czech Republic', 'India', 'Brazil', 'Greece', 'Congo', 'Senegal', 'Burkina Faso', 'Romania', 'Philippines', 'Vietnam', 'Trinidad and Tobago', 'Bulgaria', 'Chile', 'Norway', 'Kazakhstan', 'Algeria', 'Luxembourg', 'Georgia', 'Ukraine', 'Botswana', 'Aruba', 'Israel', 'Turkey', 'Ecuador', 'Lebanon', 'Morocco', 'Bosnia and Herzegovina', 'Bahamas', 'Malaysia', 'Bhutan', 'Jamaica', 'Pakistan', 'Nepal', "Cote D'Ivoire", 'Thailand', 'Namibia', 'Cameroon', 'Colombia', 'Czechoslovakia', 'Uruguay',

In [130]:
print("Número de idiomas distintos:", len(spoken_languages_unique))
print(spoken_languages_unique)

Número de idiomas distintos: 75
['English', 'Français', 'Español', 'Deutsch', 'Pусский', 'Latin', 'Nederlands', '广州话 / 廣州話', '普通话', 'Magyar', 'shqip', 'Italiano', '한국어/조선말', 'فارسی', 'Dansk', '', '日本語', 'العربية', 'Hrvatski', 'Bosanski', 'Română', 'Bahasa indonesia', 'Bahasa melayu', 'svenska', 'עִבְרִית', 'Český', 'Polski', 'Gaeilge', 'Norsk', 'Slovenčina', 'Tiếng Việt', 'Português', 'हिन्दी', 'Català', 'Íslenska', 'Afrikaans', 'Srpski', 'বাংলা', 'Wolof', 'Cymraeg', 'ภาษาไทย', 'Latviešu', 'Kiswahili', 'български език', 'ελληνικά', 'Türkçe', 'suomi', 'Esperanto', 'Український', 'ქართული', 'Bokmål', 'No Language', 'euskera', 'Azərbaycan', 'Malti', 'اردو', 'isiZulu', 'Bamanankan', 'پښتو', 'Somali', 'ਪੰਜਾਬੀ', 'беларуская мова', 'தமிழ்', 'Galego', 'Kinyarwanda', 'қазақ', 'Eesti', 'Lietuvi\x9akai', 'Slovenščina', 'తెలుగు', 'Fulfulde', '??????', '?????', 'ozbek', 'Hausa']


"Revenue" y "Budget" (Los valores nulos de estos campos deben ser rellenados por el número 0)

In [131]:
movies_aux["revenue"].isna().sum()

3

In [132]:
# Tenemos 6 nulos en "Revenue", rellenamos con 0
movies_aux["revenue"][movies_aux["revenue"].isna()] = 0

In [133]:
# Hacemos lo mismo con budget. Vemos que no hay valores nulos
movies_aux["budget"] = movies["budget"]
movies_aux["budget"].isna().sum()

3

"Release_date" (Los valores nulos deben eliminarse)

In [134]:
movies_aux["release_date"].isna().sum()

87

In [135]:
# Borramos las filas con valores nulos en release date
movies_aux.dropna(subset = "release_date", inplace = True)

De haber fechas, deberán tener el formato AAAA-mm-dd

In [11]:
# Creamos una función que transforme un string a tipo de dato datetime en el formato requerido
def parse_date(date_str):
    try:
        return pd.to_datetime(date_str, format="%Y-%m-%d")
    except ValueError:
        return pd.Timestamp('1900-01-01')

movies_aux["release_date"] = movies_aux["release_date"].apply(parse_date)       # Aplicamos la función a la columna "release_date"

In [18]:
# Vemos si se generaron datos de error al no encontrar el formato necesario
movies_aux[movies_aux["release_date"] == "1900-01-01"]

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,tagline,title,video,vote_average,vote_count,belongs_to_collection,genres,production_companies,production_countries,spoken_languages
36528,False,0.0,NaN,195557.0,tt0230613,en,Panorama of Eiffel Tower,“Showing the entire height of this wonderful s...,0.071812,/36IbMoe4Ehab7hAJiy9gxen2M7B.jpg,...,NaN,Panorama of Eiffel Tower,False,6.5,2.0,['Sin_Categoria'],['Documentary'],['Edison Manufacturing Company'],['United States of America'],['No Language']
36530,False,0.0,NaN,195535.0,tt0309973,en,Palace of Electricity,"A panoramic shot, making a full circle, at the...",0.071779,/acyoJJWlHAvAe1b1L7toYSy5sZ8.jpg,...,NaN,Palace of Electricity,False,5.5,2.0,['Sin_Categoria'],['Documentary'],['Edison Manufacturing Company'],['United States of America'],['No Language']
45364,False,0.0,NaN,49280.0,tt0135453,fr,L'Homme orchestre,A band-leader has arranged seven chairs for th...,1.109068,/ZLOgI7KjtWby1NEg2pjU2Id60W.jpg,...,NaN,The One-Man Band,False,6.5,22.0,['Sin_Categoria'],"['Fantasy', 'Action', 'Thriller']",['Star Film Company'],['France'],['No Language']
45365,False,0.0,NaN,106807.0,tt0135571,fr,Nouvelles luttes extravagantes,A series of fantastical wrestling matches.,0.225432,/tn5J3PszVETDkKj49nakmeqr8Od.jpg,...,NaN,The Fat and Lean Wrestling Match,False,6.5,6.0,['Sin_Categoria'],"['Comedy', 'Fantasy']",['Star-Film'],['France'],['No Language']


In [19]:
# Verificamos si estos datos estuvieron siempre contrastando con el dataset original y efectivamente fue así
movies[movies["release_date"] == "1900-01-01"]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
36574,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,195557,tt0230613,en,Panorama of Eiffel Tower,“Showing the entire height of this wonderful s...,...,1900-01-01,0.0,1.0,"[{'iso_639_1': 'xx', 'name': 'No Language'}]",Released,NaN,Panorama of Eiffel Tower,False,6.5,2.0
36576,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,195535,tt0309973,en,Palace of Electricity,"A panoramic shot, making a full circle, at the...",...,1900-01-01,0.0,1.0,"[{'iso_639_1': 'xx', 'name': 'No Language'}]",Released,NaN,Palace of Electricity,False,5.5,2.0
45450,False,NaN,0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...",NaN,49280,tt0135453,fr,L'Homme orchestre,A band-leader has arranged seven chairs for th...,...,1900-01-01,0.0,1.0,"[{'iso_639_1': 'xx', 'name': 'No Language'}]",Released,NaN,The One-Man Band,False,6.5,22.0
45451,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 14, 'nam...",NaN,106807,tt0135571,fr,Nouvelles luttes extravagantes,A series of fantastical wrestling matches.,...,1900-01-01,0.0,2.0,"[{'iso_639_1': 'xx', 'name': 'No Language'}]",Released,NaN,The Fat and Lean Wrestling Match,False,6.5,6.0


Deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [28]:
# Creamos la columna "release_year" con el atributo year de datetime
movies_aux["release_year"] = movies_aux["release_date"].apply(lambda x: x.year)

Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0

In [35]:
# Vemos si hay algun NaN en los campos revenue y budget
print(movies_aux["revenue"].isna().sum())
print(movies_aux["budget"].isna().sum())

0
3


In [38]:
# Hay 3 NaN en budget, limpiamos a 0
movies_aux["budget"].fillna(0, inplace = True)

In [48]:
# Definimos la nueva columna "return" y le cargamos los valores (si el denominador es 0, retorna 0)
movies_aux["return"] = np.where(movies_aux["budget"] == 0, 0, movies_aux["revenue"] / movies_aux["budget"])

Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage

In [56]:
movies_aux = movies_aux.drop(columns = ["video", "imdb_id", "adult", "original_title", "poster_path", "homepage"])

In [20]:
# Guardamos el dataframe para volver a utilizarlo luego para hacer el EDA
movies_aux.to_csv('Dataset/movies_aux.csv', index = False)

Archivo credits (información sobre actores, directores, screenplay, etc. de cada película)

In [3]:
pd.read_csv('Dataset\credits.csv').head()       # Similar a movies, hay columnas anidadas. Vamos a desanidar cast y crew por separado

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [13]:
# Cargo el archivo Credits, expandimos "cast" para que cada fila sea un character distinto
credits_cast = pd.read_csv('Dataset\credits.csv')
credits_cast.drop(columns = ["crew"], inplace = True)
credits_cast['cast'] = credits_cast['cast'].apply(ast.literal_eval)       # Convierte la columna 'cast' en una lista de diccionarios
credits_cast = credits_cast.explode('cast')       # Expande 'cast' para obtener filas separadas (un diccionario por fila)
credits_cast.dropna(inplace = True)
credits_cast = credits_cast.reset_index(drop = True)      # Reseteo el índice

In [68]:
# Hacemos lo mismo que con cast, solo que ahora expandimos "crew" (donde se aloja info sobre los directores entre otros)
credits_crew = pd.read_csv('Dataset\credits.csv')
credits_crew.drop(columns = ["cast"], inplace = True)
credits_crew['crew'] = credits_crew['crew'].apply(ast.literal_eval)       # Convierte la columna 'crew' en una lista de diccionarios
credits_crew = credits_crew.explode('crew')       # Expande 'crew' para obtener filas separadas (un diccionario por fila)
credits_crew.dropna(inplace = True)
credits_crew = credits_crew.reset_index(drop = True)      # Reseteo el índice

In [8]:
credits_cast = pd.read_csv('Dataset\credits_cast_aux.csv')
credits_cast['cast'] = credits_cast['cast'].apply(ast.literal_eval)
credits_crew = pd.read_csv('Dataset\credits_crew_aux.csv')
credits_crew['crew'] = credits_crew['crew'].apply(ast.literal_eval)

In [9]:
# Dejamos solo el campo name en el caso de cast y los campos job y name en crew (reducimos dimensiones para poder manejar mejor los archivos)
credits_cast_aux = pd.DataFrame()
credits_crew_aux = pd.DataFrame()
credits_cast_aux["cast_name"]  = credits_cast["cast"].apply(lambda x: x["name"])
credits_crew_aux["crew_name"]  = credits_crew["crew"].apply(lambda x: x["name"])
credits_crew_aux["crew_job"]  = credits_crew["crew"].apply(lambda x: x["job"])

In [10]:
# Los guardamos en csv para ser consumidos por la API
credits_cast_aux.to_csv('Dataset/credits_cast.csv', index = False)
credits_crew_aux.to_csv('Dataset/credits_crew.csv', index = False)